##### Copyright 2018 The TensorFlow Authors.

In [2]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.15.2


## Set up the layers of your keras model
In this example we setup a deep neural net. We won't seek to train the model.


In [19]:
input_shape = (4,)
model = tf.keras.Sequential([
    tf.keras.Input(shape = input_shape),
    tf.keras.layers.Lambda(lambda x: x*2), #trivial operation multiplying all inputs by a factor 2
    tf.keras.layers.Dense(32), #comment out if you want to build a trivial model that only multiplies input by 2
    tf.keras.layers.Dense(32), #comment out if you want to build a trivial model that only multiplies input by 2
    tf.keras.layers.Dense(4), #comment out if you want to build a trivial model that only multiplies input by 2
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 4)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                160       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 132       
Total params: 1,348
Trainable params: 1,348
Non-trainable params: 0
_________________________________________________________________


In [21]:
# creating a test input (ones)
x = np.array([1,1,1,1])
x = np.reshape(x, (1,4))
x.shape

(1, 4)

In [26]:
# testing the model with a test input
model.predict(x) 

array([[ 1.8745925 ,  0.5256647 ,  0.83579564, -2.0971415 ]],
      dtype=float32)

## Save model for bonsai model import
1. tf saved model format using SavedModelBuilder
2. onnx saved model
3. NOT COMPATIBLE with bonsai model import: direct tf SavedModel

In [24]:
#save keras model using SavedModelBuilder: compress as a .zip and ready for bonsai model import
from keras import backend as K
import tensorflow as tf
signature = tf.saved_model.signature_def_utils.predict_signature_def(
    inputs={"data": model.input}, outputs={"out": model.output}
)
MODEL_PATH_SAVEDMODEL = "./state_transform_deep"
builder = tf.saved_model.builder.SavedModelBuilder(MODEL_PATH_SAVEDMODEL)
print('builder')
builder.add_meta_graph_and_variables(
    sess=K.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature
    },
)
builder.save()

builder
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./state_transform_deep/saved_model.pb


b'./state_transform_deep/saved_model.pb'

In [25]:
#save keras model as ONNX: ready for bonsai model import
import os
os.environ['TF_KERAS'] = '1'
import keras2onnx
onnx_model = keras2onnx.convert_keras(model, model.name)

with open("state_transform_deep.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

The ONNX operator number change on the optimization: 14 -> 7
The maximum opset needed by this model is only 9.


In [ ]:
#CAUTION this way of saving tf SavedModel is not be compatible with bonsai model import
#save keras model as tf savedmodel
#model.save('saved_model/my_model',save_format='SavedModel')